# Tarea 4 Inteligencia Computacional: Convolutional Neural Network
#### Otoño 2017


## Introducción

Este es el código base para realizar la tarea 4 del curso de Inteligencia Computacional EL-4106. El objetivo es lograr la clasificación de las imágenes de MNIST.

En este archivo se proveen funciones auxiliares para la creación y modificación de la CNN que usará para realizar la clasificación. También busca que usted se familiarice con TensorFlow.

## Dependencias

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

ModuleNotFoundError: No module named 'tensorflow'

Este código usa Python 3.5.2 (ambiente Anaconda) y TensorFlow versión:

In [1]:
tf.__version__

NameError: name 'tf' is not defined

## Configuración por defecto de la red convolucional

La configuración macro de la red se muestra a continuación. Usted es libre de modificar estos parámetros según estime conveniente.

In [3]:
# Convolutional Layer 1.
filter_size1 = 5          # Filtros son de 5 x 5 pixeles.
num_filters1 = 16         # Hay 16 de estos filtros.

# Convolutional Layer 2.
filter_size2 = 5          # Filtros son de 5x5 pixeles.
num_filters2 = 36         # Hay 16 de estos filtros.

# Fully-connected layer.
fc_size = 128             # Número de neuronas de la capa fully-connected.

## Descarga y/o carga de la base de datos

La base de datos MNIST pesa aproximadamente 12 MB. Si no está en el directorio 'data/MNIST', será descargada automáticamente al ejecutar el siguiente bloque de código.

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


A continuación la base de datos es dividida en tres subconjuntos mutuamente excluyentes.

In [5]:
print("Tamaños de los subconjuntos de la base de datos:")
print("Training-set:\t\t{}".format(len(data.train.labels)))
print("Test-set:\t\t{}".format(len(data.test.labels)))
print("Validation-set:\t\t{}".format(len(data.validation.labels)))

Tamaños de los subconjuntos de la base de datos:
Training-set:		55000
Test-set:		10000
Validation-set:		5000


Las etiquetas de las clases están codificadas en "One-Hot". El dígito "cero" corresponde a la clase 0, el dígito "uno" a la clase 1 y así sucesivamente.Para medir el desempeño de la red se guardan previamente las etiquetas de las clases del conjunto de prueba y validación:

In [6]:
data.test.cls = np.argmax(data.test.labels, axis=1)
data.validation.cls = np.argmax(data.test.labels, axis=1)

## Dimensiones de los datos

A continuación se definen variables que caracterizan a las imágenes de la base de datos.

In [7]:
# Las imágenes de MNIST son de 28 x 28 pixeles.
img_size = 28
# Tamaño de arreglos unidimensionales que podrían guardar los datos de estas imágenes.
img_size_flat = img_size * img_size
# Tupla que sirve para redimensionar arreglos.
img_shape = (img_size, img_size)
# Número de canales de color de las imágenes. Si las imágenes fueran a color, este número sería 3.
num_channels = 1
# Número de clases.
num_classes = 10

## TensorFlow Graph

El propósito de TensorFlow es tener un "computational graph" que puede ser ejecutado de forma muy eficiente.

Un "TensorFlow graph" posee las siguientes componentes:

* "Placeholder variables", usadas para entregarle información de entrada al grafo.
* Variables que serán optimizadas para que la CNN se desempeñe mejor.
* Método de optimización para actualizar las variables.
* Función de costo que sirve para encauzar la actualización de variables.
* Formulas matemáticas que use la CNN.


### Funciones para crear variables nuevas

Sirven para que usted cree pesos y biases para su CNN.

In [8]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

### Función para crear una nueva CNN

Esta función crea una capa convolucional en el "computational graph" de Tensorflow.

Se asume que la entrada es un tensor 4-D con las siguientes dimensiones:

1. Número de imágenes.
2. Dimensión Y (cartesiana) de cada imagen.
3. Dimensión X (cartesiana) de cada imagen.
4. Canales de cada imagen.

La salida es otro tensor 4-D con las siguientes dimensiones:

1. Número de imágenes, el mismo que el de la entrada.
2. Dimensión Y (cartesiana) de cada imagen. 
3. Dimensión X (cartesiana) de cada imagen.
4. Canales producidos por los filtros convolucionales.

In [9]:
def new_conv_layer(input,              # Capa anterior.
                   num_input_channels, # Numero de canales de la capa anterior.
                   filter_size,        # Ancho y alto de cada filtro.
                   num_filters,        # Número de filtros.
                   use_pooling=True):  # Usar 2x2 max-pooling.

    # Forma de los filtros convolucionales (de acuerdo a la API de TF).
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Creación de los filtros.
    weights = new_weights(shape=shape)

    # Creación de biases, uno por filtro.
    biases = new_biases(length=num_filters)

    # Creación de la operación de convolución para TensorFlow.
    # Notar que se han configurado los strides en 1 para todas las dimensiones.
    # El primero y último stride siempre deben ser uno.
    # Si strides=[1, 2, 2, 1], entonces el filtro es movido
    # de 2 en 2 pixeles a lo largo de los ejes x e y de la imagen.
    # padding='SAME' significa que la imagen de entrada se rellena
    # con ceros para que el tamaño de la salida se mantenga.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Agregar los biases a los resultados de la convolución.
    layer += biases

    # Usar pooling para hacer down-sample de la entrada.
    if use_pooling:
        # Este es 2x2 max pooling, lo que significa que se considera
        # una ventana de 2x2 y se selecciona el valor mayor
        # de los 4 pixeles seleccionados. ksize representa las dimensiones de 
        # la ventana de pooling y el stride define cómo la ventana se mueve por la imagen.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)

    # La función retorna el resultado de la capa y los pesos aprendidos.
    return layer, weights

### Función para estirar un tensor de salida

Se usa para reducir las dimensiones del tensor de salida de la capa convolucional a uno 2D que sirva de entrada a la capa fully connected.

In [10]:
def flatten_layer(layer):
    # Obtener dimensiones de la entrada.
    layer_shape = layer.get_shape()

    # Obtener numero de características.
    num_features = layer_shape[1:4].num_elements()
    
    # Redimensionar la salida a [num_images, num_features].
    layer_flat = tf.reshape(layer, [-1, num_features])

    # Las dimensiones de la salida son ahora:
    # [num_images, img_height * img_width * num_channels]
    # Retornar
    return layer_flat, num_features

### Función para crear capa fully-connected

In [11]:
def new_fc_layer(input,          # Capa anterior.
                 num_inputs,     # Numero de entradas.
                 num_outputs,    # Numero de salidas.
                 use_relu=True): # Decide si usar ReLU o no.

    # Crear pesos y biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Evaluar capa fully connected.
    layer = tf.matmul(input, weights) + biases

    # Usar ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

### Placeholder variables

Las variables "placeholder" sirven como entradas para el "computational graph" de Tensorflow.
Primero se define una variable placeholder para las imágenes de entrada. Estas son interpretadas como "tensores" (vectores o matrices multidimensionales). El tipo de datos se configura como `float32`, y su forma se deja como `[None, img_size_flat]`, donde `None` significa que el tensor puede contener un numero arbitrario de imágenes, cada una representada como un vector de largo `img_size_flat`.

In [12]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

La red convolucional espera que `x` sea un tensor 4-D, por lo que es necesario hacer un re-shape. 
La forma del tensor debe ser `[num_images, img_height, img_width, num_channels]`.
Notar que `img_height == img_width == img_size` y `num_images` puede ser calculada, por lo que se usa -1 en la primera dimensión.

In [13]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

Análogamente, se define un placeholder para tener los valores de las etiquetas de las clases.

In [14]:
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')

También se podría tener un placeholder con el número de la clase, pero en vez de eso se va a calcular, pues se usará softmax en la capa fully connected final.

In [15]:
y_true_cls = tf.argmax(y_true, dimension=1)

### Capa de convolución 1

Creación de la primera capa de convolución. Al final se realiza submuestreo con un 2x2 max-pooling.

In [16]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

Puede revisar las dimensiones del tensor de salida como sigue:

In [17]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

### Capa de convolución 2

In [18]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

### Flatten Layer

In [19]:
layer_flat, num_features = flatten_layer(layer_conv2)

### Capa fully-connected 1

In [20]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

### Fully-Connected Layer 2

In [21]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

### Clase predicha

Se utiliza softmax para normalizar la salida, luego se toma el valor máximo.

In [22]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, dimension=1)

### Función de costo

Usamos cross-entropy.
Tensorflow la implementa de forma nativa. Como la función calcula softmax internamente, debe entregarsele la salida de la capa fully-connected 2 directamente.

In [23]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)

Lo anterior calcula cross-entropy para cada imagen. El costo será el promedio de estas mediciones.

In [24]:
cost = tf.reduce_mean(cross_entropy)

### Método de optimización

Usamos `AdamOptimizer`.
Puede cambiarlo según se pida en la tarea.

In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

### Medida de desempeño

Se obtiene un vector de booleanos que indican si la clase predicha es o no igual a la clase verdadera de cada imagen.
Luego dicho vector se niega para que True sea 1, y se calcula el promedio.

In [26]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Ejecutar TensorFlow 

### Crear TensorFlow session

Ya creado el grafo de TF, se crea una sesión para ejecutarlo.

In [27]:
session = tf.Session(config=tf.ConfigProto(log_device_placement=True))

### Inicialización de variables

In [28]:
session.run(tf.global_variables_initializer())

### Función para realizar optimización

In [29]:
# Entrenamiento realizado por batches.
train_batch_size = 100

# Contador de iteraciones.
total_iterations = 0

def optimize(num_iterations):
    
    global total_iterations

    # Tiempo de inicio
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Obtener batch de conjunto de entrenamiento.
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)

        # Se pone el batch en un diccionario asignándole nombres de las
        # variables placeholder antes definidas.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Ejecución del optimizador con los batches del diccionario.
        session.run(optimizer, feed_dict=feed_dict_train)

        # Se imprime elprogreso cada 100 iteraciones.
        if i % 50 == 0:
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Iterations: {0:>6}, Training Accuracy: {1:>6.1%}"
            print(msg.format(i, acc))

    # Actualización del número de iteraciones.
    total_iterations += num_iterations

    # Tiempo de finalización.
    end_time = time.time()

    # Tiempo transcurrido.
    time_dif = end_time - start_time


    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

### Función para mostrar desempeño en test-set

In [30]:
# Dividir test set en batches. (Usa batches mas pequeños si la RAM falla).
test_batch_size = 256

def print_test_accuracy():

    # Número de imagenes en test-set.
    num_test = len(data.test.images)

    # Crea arreglo para guardar clases predichas.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Calcular clases predichas.
    i = 0
    while i < num_test:
        
        j = min(i + test_batch_size, num_test)
        images = data.test.images[i:j, :]
        labels = data.test.labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        i = j
    
    # Labels reales.
    cls_true = data.test.cls

    # Arreglo booleano de clasificaciones correctas.
    correct = (cls_true == cls_pred)
    
    #Número de clasificaciones correctas.
    correct_sum = correct.sum()

    # Accuracy
    acc = float(correct_sum) / num_test
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

## Optimizar

Elige un número de iteraciones y entrena la CNN.

In [31]:
#Definir número de iteraciones que desea entrenar a la red
optimize(num_iterations=5500)

Iterations:      0, Training Accuracy:  12.0%
Iterations:     50, Training Accuracy:  56.0%
Iterations:    100, Training Accuracy:  78.0%
Iterations:    150, Training Accuracy:  74.0%
Iterations:    200, Training Accuracy:  85.0%
Iterations:    250, Training Accuracy:  82.0%
Iterations:    300, Training Accuracy:  93.0%
Iterations:    350, Training Accuracy:  88.0%
Iterations:    400, Training Accuracy:  84.0%
Iterations:    450, Training Accuracy:  93.0%
Iterations:    500, Training Accuracy:  94.0%
Iterations:    550, Training Accuracy:  90.0%
Iterations:    600, Training Accuracy:  97.0%
Iterations:    650, Training Accuracy:  90.0%
Iterations:    700, Training Accuracy:  91.0%
Iterations:    750, Training Accuracy:  94.0%
Iterations:    800, Training Accuracy:  92.0%
Iterations:    850, Training Accuracy:  96.0%
Iterations:    900, Training Accuracy:  95.0%
Iterations:    950, Training Accuracy:  93.0%
Time usage: 0:00:27


In [32]:
print_test_accuracy()

Accuracy on Test-Set: 94.0% (9399 / 10000)


### Cerrar TensorFlow Session

In [33]:
# Si usted ejecuta esta linea de código debe cerrar el notebook y reiniciarlo.
# Es solo para informar como liberar los recursos que ocupa TF.
# session.close()